In [1]:
import numpy as np
import nibabel as nib
from dipy.tracking.local import LocalTracking, ThresholdTissueClassifier
from dipy.tracking.utils import random_seeds_from_mask
from dipy.reconst.dti import TensorModel
from dipy.reconst.csdeconv import (ConstrainedSphericalDeconvModel,
                                   auto_response)
from dipy.direction import peaks_from_model
from dipy.data import fetch_stanford_hardi, read_stanford_hardi, get_sphere
from dipy.segment.mask import median_otsu
from dipy.viz import actor, window
from dipy.io.image import save_nifti
from nibabel.streamlines import save as save_trk
from nibabel.streamlines import Tractogram
from dipy.tracking.streamline import Streamlines

In [4]:
fimg = ("/space/hemera/1/users/cmaffei/scilpy_mrtrix_comparison/1000/from_mgh/data_con.nii.gz")
img = nib.load(fimg)
fbval=("/space/hemera/1/users/cmaffei/scilpy_mrtrix_comparison/1000/from_mgh/bval.txt")
fbvec = ("/space/hemera/1/users/cmaffei/scilpy_mrtrix_comparison/1000/from_mgh/scilpy/bvec-x.txt")
data = img.get_data()
from dipy.io import read_bvals_bvecs
bvals, bvecs = read_bvals_bvecs(fbval, fbvec)
from dipy.core.gradients import gradient_table
gtab = gradient_table(bvals,bvecs)
print(gtab.bvals)
print('data.shape (%d, %d, %d, %d)' % data.shape)
affine = img.affine

[    0.     0.     0.     0.     0.  1000.  1000.  1000.  1000.  1000.
  1000.  1000.  1000.  1000.  1000.  1000.  1000.  1000.  1000.  1000.
  1000.  1000.  1000.  1000.  1000.  1000.  1000.  1000.  1000.  1000.
  1000.  1000.  1000.  1000.  1000.  1000.  1000.  1000.  1000.  1000.
  1000.  1000.  1000.  1000.  1000.  1000.  1000.  1000.  1000.  1000.
  1000.  1000.  1000.  1000.  1000.  1000.  1000.  1000.  1000.  1000.
  1000.  1000.  1000.  1000.  1000.  1000.  1000.  1000.  1000.]
data.shape (140, 140, 96, 69)


In [9]:
maskdata, mask = median_otsu(data, 3, 1, False,
                             vol_idx=range(10, 50), dilate=2)
save_nifti('/space/hemera/1/users/cmaffei/scilpy_mrtrix_comparison/1000/from_mgh/mask.nii.gz',
          mask.astype('float32'), img.affine)
print('mask.shape (%d, %d, %d)' % mask.shape)

mask.shape (140, 140, 96)


In [10]:
response, ratio = auto_response(gtab, data, roi_radius=10, fa_thr=0.7)

csd_model = ConstrainedSphericalDeconvModel(gtab, response)
sphere = get_sphere('symmetric724')

csd_peaks = peaks_from_model(model=csd_model,
                             data=data,
                             sphere=sphere,
                             mask=mask,
                             relative_peak_threshold=.5,
                             min_separation_angle=25,
                             parallel=True)

/autofs/space/tinia_001/users/chiara/anaconda2/lib/python2.7/site-packages/dipy/reconst/csdeconv.py:588: UserWarning: maximum number of iterations exceeded - failed to converge
  warnings.warn(msg)
/autofs/space/tinia_001/users/chiara/anaconda2/lib/python2.7/site-packages/dipy/reconst/csdeconv.py:588: UserWarning: maximum number of iterations exceeded - failed to converge
  warnings.warn(msg)
/autofs/space/tinia_001/users/chiara/anaconda2/lib/python2.7/site-packages/dipy/reconst/csdeconv.py:588: UserWarning: maximum number of iterations exceeded - failed to converge
  warnings.warn(msg)


In [40]:
tensor_model = TensorModel(gtab, fit_method='WLS')
tensor_fit = tensor_model.fit(data, mask)

fa = tensor_fit.fa

save_nifti('/space/hemera/1/users/cmaffei/scilpy_mrtrix_comparison/1000/from_mgh/fa_map.nii.gz', fa, img.affine)

In [139]:
tissue_classifier = ThresholdTissueClassifier(fa, 0.1)
seeds = random_seeds_from_mask(fa > 0.3, seeds_count=1)

##important noote: always track in world space!!!! no affine provided when placing the seeds!

In [142]:
interactive = True
ren = window.Renderer()
ren.add(actor.peak_slicer(csd_peaks.peak_dirs,
                          csd_peaks.peak_values,
                          colors=None))

if interactive:
    window.show(ren, size=(900, 900))
else:
    window.record(ren, out_path='csd_direction_field.png', size=(900, 900))

In [140]:
streamline_generator = LocalTracking(csd_peaks, tissue_classifier,
                                     seeds, affine=np.eye(4),
                                     step_size=0.5)

streamlines = Streamlines(streamline_generator)

In [53]:
print(len(streamlines))


261913


In [174]:
sphere.vertices
from dipy.data import get_sphere
sphere_642 = get_sphere('symmetric642')
sphere_642.vertices.dtype

dtype('float32')

In [18]:
interactive = True
ren.clear()
ren.add(actor.line(streamlines))

if interactive:
    window.show(ren, size=(900, 900))
else:
    print('Saving illustration as det_streamlines.png')
    window.record(ren, out_path='det_streamlines.png', size=(900, 900))

In [54]:
from dipy.io.trackvis import save_trk
streamlines = Streamlines(streamline_generator)
save_trk('/space/hemera/1/users/cmaffei/scilpy_mrtrix_comparison/1000/from_mgh/det_streamlines_23_5_shape2.trk', streamlines, img.affine, fa.shape)

/autofs/space/tinia_001/users/chiara/anaconda2/lib/python2.7/site-packages/dipy/io/trackvis.py:22: DeprecationWarning: The `dipy.io.trackvis.save_trk` function is deprecated as of version 0.14 of Dipy and will be removed in a future version. Please use `dipy.io.streamline.save_trk` function instead
  DeprecationWarning)


teh following way saves the file trk correctly with the trasform!

In [141]:
from dipy.io.streamline import save_trk

fname= '/space/hemera/1/users/cmaffei/scilpy_mrtrix_comparison/1000/from_mgh/det_streamlines_23_5_shape.trk'
voxel_size = voxel_size = img.header.get_zooms()[:3]
save_trk(fname, streamlines, img.affine, shape=img.shape[:3], vox_size=voxel_size)